In [1]:
import pandas as pd
import re

In [2]:
# df = pd.read_csv('NOTEEVENTS_first10K.csv',engine='python',error_bad_lines=False)
# df.head()

In [4]:
df_diabetes = pd.read_csv('../data/outfile.csv',header=None)
df_diabetes.head()

,0,1,2,3,4,5,6,7,8,9,10,11
0,25,236,26601,116471,2127-07-30,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2127-7-21**] ...
1,32,243,2414,106238,2186-06-11,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2186-6-7**] Discharge ...
2,41,188,98046,139402,2198-08-04,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2198-7-17**] ...
3,47,194,22180,116189,2132-09-09,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2132-9-4**] D...
4,53,200,9487,112385,2138-12-12,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2138-12-2**] Discharge...


In [5]:
df_diabetes.columns = ['index','ROW_ID','SUBJECT_ID','HADM_ID','CHARTDATE','CHARTTIME','STORETIME',
                                                       'CATEGORY','DESCRIPTION','CGID','ISERROR','TEXT']

In [6]:
df_diabetes.head()

,index,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT
0,25,236,26601,116471,2127-07-30,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2127-7-21**] ...
1,32,243,2414,106238,2186-06-11,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2186-6-7**] Discharge ...
2,41,188,98046,139402,2198-08-04,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2198-7-17**] ...
3,47,194,22180,116189,2132-09-09,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2132-9-4**] D...
4,53,200,9487,112385,2138-12-12,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2138-12-2**] Discharge...


In [7]:
df_diabetes.shape

(186654, 12)

In [8]:
df_diabetes['CATEGORY'].value_counts()

Nursing/other        53830
Radiology            45497
Nursing              29188
ECG                  20592
Physician            17054
Discharge summary     7421
Echo                  5041
Respiratory           4534
Nutrition             1262
General                944
Rehab Services         909
Social Work            241
Case Management        111
Consult                 17
Pharmacy                13
Name: CATEGORY, dtype: int64

In [9]:
hadm_list = list(df_diabetes['HADM_ID'].unique())

In [10]:
len(hadm_list)

6588

In [12]:
admit_list = pd.read_csv('../data/addmision_id.csv',header=None)
addmision_id=list(admit_list[1])

In [13]:
len(addmision_id)

6601

In [14]:
# the dataframe of all discharge summary, some patients have more than one 
df_dia_1 = df_diabetes[df_diabetes['CATEGORY']=='Discharge summary']

In [15]:
df_dia_1

,index,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT
0,25,236,26601,116471,2127-07-30,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2127-7-21**] ...
1,32,243,2414,106238,2186-06-11,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2186-6-7**] Discharge ...
2,41,188,98046,139402,2198-08-04,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2198-7-17**] ...
3,47,194,22180,116189,2132-09-09,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2132-9-4**] D...
4,53,200,9487,112385,2138-12-12,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2138-12-2**] Discharge...
...,...,...,...,...,...,...,...,...,...,...,...,...
7469,60388,59651,49879,114661,2125-08-17,NaN,NaN,Discharge summary,Addendum,NaN,NaN,"Name: [**Known lastname 18448**],[**Known fir..."
7470,60389,59652,77532,157093,2102-08-24,NaN,NaN,Discharge summary,Addendum,NaN,NaN,"Name: [**Known lastname 18453**],[**Known fir..."
7471,60401,59620,40866,112538,2104-07-15,NaN,NaN,Discharge summary,Addendum,NaN,NaN,"Name: [**Known lastname 18353**],[**Known fir..."
7473,60411,59608,93924,135672,2104-06-14,NaN,NaN,Discharge summary,Addendum,NaN,NaN,"Name: [**Known lastname 18321**],[**Known fir..."


In [16]:
df_dia_1 = df_dia_1[['SUBJECT_ID','HADM_ID','CHARTDATE','CATEGORY','TEXT']]

In [17]:
df_dia_1['CATEGORY'].value_counts()

Discharge summary    7421
Name: CATEGORY, dtype: int64

In [18]:
df_dia_1['text_list']=df_dia_1['TEXT'].map(lambda x: re.sub(r'[^\w]', ' ', x)).map(lambda x: re.sub(" \d+", " ",x))

/Users/susanchen/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [19]:
df_dia_1.head()

,SUBJECT_ID,HADM_ID,CHARTDATE,CATEGORY,TEXT,text_list
0,26601,116471,2127-07-30,Discharge summary,Admission Date: [**2127-7-21**] ...,Admission Date Dischar...
1,2414,106238,2186-06-11,Discharge summary,Admission Date: [**2186-6-7**] Discharge ...,Admission Date Discharge Date ...
2,98046,139402,2198-08-04,Discharge summary,Admission Date: [**2198-7-17**] ...,Admission Date Dischar...
3,22180,116189,2132-09-09,Discharge summary,Admission Date: [**2132-9-4**] D...,Admission Date Dischar...
4,9487,112385,2138-12-12,Discharge summary,Admission Date: [**2138-12-2**] Discharge...,Admission Date Discharge Date ...


In [21]:
df_dia_1.sort_values('HADM_ID')

,SUBJECT_ID,HADM_ID,CHARTDATE,CATEGORY,TEXT,text_list
3129,533,100009,2162-05-21,Discharge summary,Admission Date: [**2162-5-16**] ...,Admission Date Dischar...
5796,51495,100034,2164-04-27,Discharge summary,Admission Date: [**2164-4-23**] ...,Admission Date Dischar...
1655,1569,100045,2176-02-15,Discharge summary,Admission Date: [**2176-2-5**] D...,Admission Date Dischar...
2144,20777,100059,2198-02-19,Discharge summary,Admission Date: [**2198-2-1**] Discharg...,Admission Date Discharge Date...
4877,82091,100099,2118-05-19,Discharge summary,Admission Date: [**2118-5-7**] D...,Admission Date Dischar...
...,...,...,...,...,...,...
7069,22711,199952,2134-04-06,Discharge summary,"Name: [**Known lastname 10675**],[**Known fir...",Name Known lastname Known firstnam...
3068,22711,199952,2134-03-22,Discharge summary,Unit No: [**Numeric Identifier 57430**]\nAdmi...,Unit No Numeric Identifier Admission ...
3114,5478,199963,2101-09-08,Discharge summary,Admission Date: [**2101-8-23**] ...,Admission Date Dischar...
6493,42131,199967,2184-08-10,Discharge summary,Admission Date: [**2184-7-31**] ...,Admission Date Dischar...


In [22]:
df_dia_1[df_dia_1.duplicated('HADM_ID')]

,SUBJECT_ID,HADM_ID,CHARTDATE,CATEGORY,TEXT,text_list
55,5544,105241,2114-04-13,Discharge summary,Admission Date: [**2114-4-3**] Discharge ...,Admission Date Discharge Date ...
81,41795,138132,2145-07-20,Discharge summary,Admission Date: [**2145-7-7**] D...,Admission Date Dischar...
127,14911,111700,2131-09-03,Discharge summary,Admission Date: [**2131-8-23**] Dischar...,Admission Date Discharge Date...
160,13739,172168,2110-02-27,Discharge summary,Admission Date: [**2110-2-17**] Dischar...,Admission Date Discharge Date...
176,19475,128816,2190-04-21,Discharge summary,Admission Date: [**2190-4-1**] Discharg...,Admission Date Discharge Date...
...,...,...,...,...,...,...
7469,49879,114661,2125-08-17,Discharge summary,"Name: [**Known lastname 18448**],[**Known fir...",Name Known lastname Known firstnam...
7470,77532,157093,2102-08-24,Discharge summary,"Name: [**Known lastname 18453**],[**Known fir...",Name Known lastname Known firstnam...
7471,40866,112538,2104-07-15,Discharge summary,"Name: [**Known lastname 18353**],[**Known fir...",Name Known lastname Known firstnam...
7473,93924,135672,2104-06-14,Discharge summary,"Name: [**Known lastname 18321**],[**Known fir...",Name Known lastname Known firstnam...


In [23]:
id_list=pd.DataFrame(df_dia_1['HADM_ID'].unique())
len(id_list)

6483

In [24]:
id_list.to_csv('discharge_id_list.csv')

In [25]:
df_dia_1.shape

(7421, 6)

In [26]:
df_final = df_dia_1.copy()

In [27]:
df_final = pd.DataFrame(df_final.groupby('HADM_ID')['text_list'].apply(lambda x: ' '.join(x)),)

In [28]:
df_final.reset_index(inplace=True)

In [29]:
df_final.head()

,HADM_ID,text_list
0,100009,Admission Date Dischar...
1,100034,Admission Date Dischar...
2,100045,Admission Date Dischar...
3,100059,Admission Date Discharge Date...
4,100099,Admission Date Dischar...


In [34]:
df_id_label = pd.read_csv('../data/MIMIC_id_label.csv')
df_id_label.drop(columns='Unnamed: 0',inplace=True)
df_id_label.head()
df_id_label.sort_values('HADM_ID')

,HADM_ID,label
69,100009,0
4017,100034,0
148,100045,0
2198,100059,0
5688,100099,0
...,...,...
1222,199923,0
2490,199952,0
474,199963,0
3599,199967,0


In [35]:
df_merge = df_final.merge(df_id_label, left_on='HADM_ID', right_on='HADM_ID')
df_merge.shape

(6483, 3)

In [36]:
df_merge.head()

,HADM_ID,text_list,label
0,100009,Admission Date Dischar...,0
1,100034,Admission Date Dischar...,0
2,100045,Admission Date Dischar...,0
3,100059,Admission Date Discharge Date...,0
4,100099,Admission Date Dischar...,0


In [ ]:
df_merge.to_csv('MIMIC_train_test.csv')

In [ ]:
# from nltk.corpus import stopwords
# s=set(stopwords.words('english'))
# s.update({'Admission','Date','Last','Name','Discharge','Completed','by','NamePattern1','NamePattern2','AM','PM'})
# len(s)

# word_list = []
# words = filter(lambda w: not w in s,text3)
# for word in words:
#     word_list.append(word)